# Show theoretical versus actual TCR

In [ ]:
import os

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from fair import __version__
from fair.energy_balance_model import multi_ebm
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import xarray as xr

In [ ]:
load_dotenv(override=True)
datadir = os.getenv("DATADIR")
os.makedirs(datadir, exist_ok=True)

In [ ]:
pl.style.use('../defaults.mplstyle')

## run 1pct

In [ ]:
output_ensemble_size=841

In [ ]:
scenarios = ['1pctCO2']

In [ ]:
df_configs_141 = pd.read_csv(
    "../data/calibration/v1.4.1/calibrated_constrained_parameters.csv",
    index_col=0,
)
df_configs_141.columns

In [ ]:
configs = list(range(output_ensemble_size))

In [ ]:
species, properties = read_properties()

In [ ]:
da_concentration = xr.load_dataarray(
    "../data/concentrations/1pctCO2_concentration_1850-1990.nc"
)

In [ ]:
f = FAIR()
f.define_time(1850, 1990, 1)
f.define_scenarios(scenarios)
species = ["CO2", "CH4", "N2O"]
properties = {
    "CO2": {
        "type": "co2",
        "input_mode": "concentration",
        "greenhouse_gas": True,
        "aerosol_chemistry_from_emissions": False,
        "aerosol_chemistry_from_concentration": False,
    },
    "CH4": {
        "type": "ch4",
        "input_mode": "concentration",
        "greenhouse_gas": True,
        "aerosol_chemistry_from_emissions": False,
        "aerosol_chemistry_from_concentration": False,
    },
    "N2O": {
        "type": "n2o",
        "input_mode": "concentration",
        "greenhouse_gas": True,
        "aerosol_chemistry_from_emissions": False,
        "aerosol_chemistry_from_concentration": False,
    },
}
f.define_configs(configs)
f.define_species(species, properties)
f.allocate()

In [ ]:
da = da_concentration.loc[dict(config="unspecified", scenario="1pctCO2")]
fe = da.expand_dims(dim=["scenario", "config"], axis=(1, 2))
f.concentration = fe.drop_vars("config") * np.ones((1, 1, output_ensemble_size, 1))

In [ ]:
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs_141.loc[:, "clim_c1":"clim_c3"].values
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs_141.loc[:, "clim_kappa1":"clim_kappa3"].values
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs_141["clim_epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs_141["clim_gamma"].values.squeeze())
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["forcing_4co2"], df_configs_141["clim_F_4xCO2"].values.squeeze())

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs_141["cc_r0"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_airborne"], df_configs_141["cc_rA"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_uptake"], df_configs_141["cc_rU"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_temperature"], df_configs_141["cc_rT"].values.squeeze(), specie="CO2")

# forcing scaling
fill(f.species_configs["forcing_scale"], df_configs_141["fscale_CO2"].values.squeeze(), specie="CO2")
fill(f.species_configs["forcing_scale"], df_configs_141["fscale_CH4"].values.squeeze(), specie="CH4")
fill(f.species_configs["forcing_scale"], df_configs_141["fscale_N2O"].values.squeeze(), specie="N2O")

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs["baseline_concentration"], 284.3169988, specie="CO2")
fill(f.species_configs["baseline_concentration"], 808.2490285, specie="CH4")
fill(f.species_configs["baseline_concentration"], 273.021047, specie="N2O")

fill(
    f.species_configs["forcing_reference_concentration"], 284.3169988, specie="CO2"
)
fill(
    f.species_configs["forcing_reference_concentration"], 808.2490285, specie="CH4"
)
fill(f.species_configs["forcing_reference_concentration"], 273.021047, specie="N2O")

# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
tcr_yr70 = f.temperature[70, 0, :, 0]

## abrupt 4xCO2

In [ ]:
f = FAIR()
f.define_time(0, 4000, 1)
f.define_scenarios(scenarios)
species = ["CO2", "CH4", "N2O"]
properties = {
    "CO2": {
        "type": "co2",
        "input_mode": "concentration",
        "greenhouse_gas": True,
        "aerosol_chemistry_from_emissions": False,
        "aerosol_chemistry_from_concentration": False,
    },
    "CH4": {
        "type": "ch4",
        "input_mode": "concentration",
        "greenhouse_gas": True,
        "aerosol_chemistry_from_emissions": False,
        "aerosol_chemistry_from_concentration": False,
    },
    "N2O": {
        "type": "n2o",
        "input_mode": "concentration",
        "greenhouse_gas": True,
        "aerosol_chemistry_from_emissions": False,
        "aerosol_chemistry_from_concentration": False,
    },
}
f.define_configs(configs)
f.define_species(species, properties)
f.allocate()

In [ ]:
f.concentration[:, 0, :, 0] = 284.3169988 * 2
f.concentration[:, 0, :, 1] = 808.2490285
f.concentration[:, 0, :, 2] = 273.021047

In [ ]:
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs_141.loc[:, "clim_c1":"clim_c3"].values
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs_141.loc[:, "clim_kappa1":"clim_kappa3"].values
)
fill(f.climate_configs["deep_ocean_efficacy"], df_configs_141["clim_epsilon"].values.squeeze())
fill(f.climate_configs["gamma_autocorrelation"], df_configs_141["clim_gamma"].values.squeeze())
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["forcing_4co2"], df_configs_141["clim_F_4xCO2"].values.squeeze())

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs_141["cc_r0"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_airborne"], df_configs_141["cc_rA"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_uptake"], df_configs_141["cc_rU"].values.squeeze(), specie="CO2")
fill(f.species_configs["iirf_temperature"], df_configs_141["cc_rT"].values.squeeze(), specie="CO2")

# forcing scaling
fill(f.species_configs["forcing_scale"], df_configs_141["fscale_CO2"].values.squeeze(), specie="CO2")
fill(f.species_configs["forcing_scale"], df_configs_141["fscale_CH4"].values.squeeze(), specie="CH4")
fill(f.species_configs["forcing_scale"], df_configs_141["fscale_N2O"].values.squeeze(), specie="N2O")

# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(f.species_configs["baseline_concentration"], 284.3169988, specie="CO2")
fill(f.species_configs["baseline_concentration"], 808.2490285, specie="CH4")
fill(f.species_configs["baseline_concentration"], 273.021047, specie="N2O")

fill(
    f.species_configs["forcing_reference_concentration"], 284.3169988, specie="CO2"
)
fill(
    f.species_configs["forcing_reference_concentration"], 808.2490285, specie="CH4"
)
fill(f.species_configs["forcing_reference_concentration"], 273.021047, specie="N2O")

# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

In [ ]:
ecs_yr4000 = f.temperature[4000, 0, :, 0]

In [ ]:
ebms = multi_ebm(
    configs,
    ocean_heat_capacity=df_configs_141.loc[:, "clim_c1":"clim_c3"].values,
    ocean_heat_transfer=df_configs_141.loc[:, "clim_kappa1":"clim_kappa3"].values,
    deep_ocean_efficacy=df_configs_141["clim_epsilon"].values.squeeze(),
    gamma_autocorrelation=df_configs_141["clim_gamma"].values.squeeze(),
    forcing_4co2=df_configs_141["clim_F_4xCO2"].values.squeeze(),
    stochastic_run=[False]*841,
    sigma_eta=df_configs_141["clim_sigma_eta"].values.squeeze(),
    sigma_xi=df_configs_141["clim_sigma_xi"].values.squeeze(),
    seed=12,
    use_seed=[False]*841,
    timestep=1,
    timebounds=[0, 1]
)

In [ ]:
ebms.ecs

In [ ]:
fig, ax = pl.subplots(1, 2, figsize=(18/2.54, 9/2.54))
ax[0].scatter(ebms.ecs, ecs_yr4000)
ax[0].plot(np.linspace(1, 8), np.linspace(1, 8), color='k')
ax[0].set_xlim(1, 8)
ax[0].set_ylim(1, 8)
ax[0].set_xlabel('Theoretical ECS (°C)')
ax[0].set_ylabel(r'abrupt-2$\times$CO$_2$ warming year 4000 (°C)')
ax[0].set_title('(a) Equilibrium climate sensitivity')
ax[1].scatter(ebms.tcr, tcr_yr70)
ax[1].plot(np.linspace(0.9, 3.5), np.linspace(0.9, 3.5), color='k')
ax[1].set_xlim(0.9, 3.5)
ax[1].set_ylim(0.9, 3.5)
ax[1].set_xlabel('Theoretical TCR (°C)')
ax[1].set_ylabel(r'1pctCO$_2$ warming year 70 (°C)')
ax[1].set_title('(b) Transient climate response')
fig.tight_layout()
pl.savefig('../plots/ebm-compare.png')
pl.savefig('../plots/ebm-compare.pdf')